<a href="https://colab.research.google.com/github/kokorikos212/Algorithms/blob/main/Elytis_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

from transformers import pipeline

In [2]:
import tensorflow as tf

In [3]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert/distilbert-base-multilingual-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

In [4]:
unmasker = pipeline('fill-mask', model='distilbert-base-multilingual-cased')
unmasker("Καλησπέρα είμαι ένα  [MASK] μηχανικής μάθησης.")

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

[{'score': 0.5381671786308289,
  'token': 10470,
  'token_str': 'της',
  'sequence': 'Καλησπέρα είμαι ένα της μηχανικής μάθησης.'},
 {'score': 0.12072676420211792,
  'token': 46447,
  'token_str': 'μιας',
  'sequence': 'Καλησπέρα είμαι ένα μιας μηχανικής μάθησης.'},
 {'score': 0.058107584714889526,
  'token': 90167,
  'token_str': 'είδος',
  'sequence': 'Καλησπέρα είμαι ένα είδος μηχανικής μάθησης.'},
 {'score': 0.018670862540602684,
  'token': 31111,
  'token_str': 'μέρος',
  'sequence': 'Καλησπέρα είμαι ένα μέρος μηχανικής μάθησης.'},
 {'score': 0.013995717279613018,
  'token': 53902,
  'token_str': 'μη',
  'sequence': 'Καλησπέρα είμαι ένα μη μηχανικής μάθησης.'}]

In [5]:
def prepare_dataset_for_fine_tuning(poems_path, max_sequence_length=50, tokenizer=None, batch_size=32, shuffle_buffer_size=10000):
    """
    Prepare a TensorFlow Dataset for fine-tuning a language model on a single text containing poems.

    Args:
    - poems_path (str): Path for the text file containing all the poems.
    - max_sequence_length (int): Maximum length of each input sequence.
    - tokenizer (Tokenizer): Tokenizer instance for tokenizing the text.
    - batch_size (int): Batch size for training.
    - shuffle_buffer_size (int): Buffer size for shuffling the dataset.

    Returns:
    - TensorFlow Dataset containing input-output pairs for fine-tuning.
    """
    if tokenizer is None:
        tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

    with open(poems_path, 'r', encoding='utf-8') as f:
        poems_text = f.read()

    # Split text into sentences
    sentences = poems_text.split('.')

    # Tokenize each sentence
    tokenized_sentences = [tokenizer.encode(sentence) for sentence in sentences if sentence.strip()]

    # Generate input-output pairs with sliding window approach
    input_sequences = []
    output_sequences = []
    for i in range(len(tokenized_sentences) - 1):
        input_sequence = tokenized_sentences[i]
        output_sequence = tokenized_sentences[i + 1]

        # Flatten the sequences to ensure they don't exceed max_sequence_length
        input_sequence = input_sequence[:max_sequence_length]
        output_sequence = output_sequence[:max_sequence_length]

        # Ensure sequences are exactly max_sequence_length long
        if len(input_sequence) == max_sequence_length and len(output_sequence) == max_sequence_length:
            input_sequences.append(input_sequence)
            output_sequences.append(output_sequence)

    # Check if input_sequences is empty
    if not input_sequences:
        raise ValueError("No sequences were generated. Please check your input text and tokenizer settings.")

    # Convert lists to tensors
    input_sequences = tf.ragged.constant(input_sequences).to_tensor()
    output_sequences = tf.ragged.constant(output_sequences).to_tensor()

    # Create TensorFlow Dataset from input-output pairs
    dataset = tf.data.Dataset.from_tensor_slices((input_sequences, output_sequences))

    # Shuffle and batch the dataset
    buffer_size = max(len(input_sequences), shuffle_buffer_size)
    dataset = dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset


In [7]:
poems_path = "cleaned_combined_Elytis.txt"
dataset = prepare_dataset_for_fine_tuning(poems_path)

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


In [8]:
def split_dataset(dataset, train_split=0.8, val_split=0.1, test_split=0.1):
    """
    Split a TensorFlow Dataset into training, validation, and test sets.

    Args:
    - dataset (tf.data.Dataset): The TensorFlow Dataset to split.
    - train_split (float): Proportion of data to be used for training.
    - val_split (float): Proportion of data to be used for validation.
    - test_split (float): Proportion of data to be used for testing.

    Returns:
    - train_dataset (tf.data.Dataset): The training dataset.
    - val_dataset (tf.data.Dataset): The validation dataset (or None if not used).
    - test_dataset (tf.data.Dataset): The testing dataset.
    """
    # Validate the split ratios
    assert train_split + val_split + test_split == 1, "Splits must sum to 1"

    # Determine the sizes of each split
    dataset_size = len(list(dataset))
    train_size = int(train_split * dataset_size)
    val_size = int(val_split * dataset_size)
    test_size = dataset_size - train_size - val_size

    # Split the dataset
    train_dataset = dataset.take(train_size)
    remaining = dataset.skip(train_size)
    val_dataset = remaining.take(val_size) if val_split > 0 else None
    test_dataset = remaining.skip(val_size)

    return train_dataset, val_dataset, test_dataset

train_set , validation_set, test_set = split_dataset(dataset)

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# dataset = train_dataset
# num_batches = 5
# for batch_num, (input_batch, output_batch) in enumerate(dataset.take(num_batches)):
#     print(f"Batch {batch_num + 1}/{num_batches}")
#     print("Input batch:")
    # print(input_batch.numpy())
    # print("Output batch:")
    # print(output_batch.numpy())

In [10]:
for batch in train_set.take(1):
    input_ids, labels = batch
    print("Batch structure:")
    print(f"Input IDs shape: {input_ids.shape}")
    print(f"Labels shape: {labels.shape}")

Batch structure:
Input IDs shape: (32, 50)
Labels shape: (32, 50)


In [ ]:
# !pip install evaluate
# !pip install --upgrade pyarrow
# !pip uninstall pyarrow datasets evaluate

!pip install pyarrow==12.0.0
!pip install datasets==2.4.0
!pip install evaluate==0.2.2

# !pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 16.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 12.0.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 11.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 789.3 kB/s eta 0:00:00


In [18]:
# !pip install accelerate -U
# !pip install bitsandbytes
# !pip install pytorch-accelerated

In [22]:
# !pip show accelerate
# !pip show transformers


In [23]:
# !pip install transformers[torch]

In [11]:
def compute_accuracy(preds, labels):
    """
    Compute the accuracy of predictions.

    Args:
    - preds: Predictions from the model.
    - labels: True labels.

    Returns:
    - accuracy: Computed accuracy.
    """
    return (preds == labels).astype(float).mean()

In [21]:
from transformers import Trainer, TrainingArguments
from accelerate import Accelerator
from transformers import pipeline
# import evaluate

from transformers import DataCollatorWithPadding

# Define the data collator
data_collator = DataCollatorWithPadding(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./test-trainer",
    num_train_epochs=1,
    evaluation_strategy="epoch",
    weight_decay=5e-4,
    logging_dir="./logs",
    logging_steps=10,  # Adjust as needed
    save_strategy="no",  # Don't save checkpoints during training
    report_to="none"
)

accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
evaluation_result = trainer.evaluate()
print("Evaluation result:", evaluation_result)


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoTokenizer, TFAutoModelForCausalLM
from accelerate import Accelerator
from transformers import pipeline
import tensorflow as tf

# Assuming tokenizer and model are already loaded
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")
model = TFAutoModelForCausalLM.from_pretrained("distilbert-base-multilingual-cased")

# Define the data collator
data_collator = DataCollatorWithPadding(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./test-trainer",
    num_train_epochs=1,
    evaluation_strategy="epoch",
    weight_decay=5e-4,
    logging_dir="./logs",
    logging_steps=10,  # Adjust as needed
    save_strategy="no",  # Don't save checkpoints during training
    report_to="none"
)

# Custom compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    accuracy = compute_accuracy(preds, labels)
    return {"accuracy": accuracy}

# Prepare your dataset using the function provided earlier
dataset = prepare_dataset_for_fine_tuning(poems_path='path_to_poems.txt', max_sequence_length=50)

# Split the dataset into training, validation, and test sets
train_dataset, val_dataset, test_dataset = split_dataset(dataset, train_split=0.8, val_split=0.1, test_split=0.1)

# Convert TensorFlow datasets to Hugging Face datasets
def tf_dataset_to_hf_dataset(tf_dataset):
    return datasets.Dataset.from_dict(
        {k: [x[k].numpy() for x in tf_dataset] for k in tf_dataset.element_spec.keys()}
    )

hf_train_dataset = tf_dataset_to_hf_dataset(train_dataset)
hf_val_dataset = tf_dataset_to_hf_dataset(val_dataset)
hf_test_dataset = tf_dataset_to_hf_dataset(test_dataset)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
evaluation_result = trainer.evaluate(eval_dataset=hf_test_dataset)
print("Evaluation result:", evaluation_result)


ValueError: Unrecognized configuration class <class 'transformers.models.distilbert.configuration_distilbert.DistilBertConfig'> for this kind of AutoModel: TFAutoModelForCausalLM.
Model type should be one of BertConfig, CamembertConfig, CTRLConfig, GPT2Config, GPT2Config, GPTJConfig, OpenAIGPTConfig, OPTConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoFormerConfig, TransfoXLConfig, XGLMConfig, XLMConfig, XLMRobertaConfig, XLNetConfig.

In [ ]:
y# Define the pipeline for masked language modeling
mlm_pipeline = pipeline("fill-mask", model=model, tokenizer=tokenizer)

# Evaluate the model on a validation set
results = mlm_pipeline.evaluate(val_dataset)
print(results)

# Save the fine-tuned model
model.save_pretrained("fine_tuned_Elytis_model")